## 원본 데이터로부터 모델 학습에 사용할 수 있는 데이터를 추출해보자.

프로세싱 순서는 아래와 같다.

1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 

2. sample rate를 16000으로 통일한다.

3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.

4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.

5. 음성이 5초 단위로 분할하여 모두 동일한 길이를 가지도록 조정한다.

6. 음성의 노이즈를 제거한다.

7. 음성의 유사도를 분석하여 가장 유사한 n개의 데이터만 사용한다.

8. 모델의 학습 데이터로 사용한다.

> 4번 과정 대신 get_sample_data를 통해 임의로 n개의 파일을 선택해 학습을 수행하는 방법 또한 있다.

In [1]:
# Load packages
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# Set project path : 본인의 프로젝트가 있는 절대 경로를 append를 통해 추가한다.
main_path = '/Users/jaewone/developer/tensorflow/baby-cry-classification'

sys.path.append(main_path)
data_path = os.path.join(main_path, 'data')
sample_data_path = os.path.join(data_path, 'sample_data')
csv_path = os.path.join(main_path, 'info.csv')

In [3]:
# Load custon packages
from utils.os import *
from utils.sound import *
from trans_data.bit_sampling import *
from trans_data.get_state_list import get_state_list_from_dir_name
from trans_data.create_state_folder import create_state_folder
from trans_data.rename_by_state import rename_files_by_state
from trans_data.trim_white_noise import trim_audio
from trans_data.split_audio import split_audio_list_in_sec

In [4]:
# 1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 
# 2. sample rate를 16000으로 통일한다.
create_state_folder(main_path, csv_path)

전체 3688개의 데이터 중 사용할 수 없는 216개의 데이터가 제외되었다.


In [5]:
# 3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.
file_list = rename_files_by_state(data_path)

In [6]:
# 4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.
failed_file_list = []
for i in tqdm(range(len(file_list))):
    try:
        trim_audio(file_list[i], inplace=True)
    except:
        failed_file_list.append(file_list[i])

print(f'trimming에 실패한 {len(failed_file_list)}개의 파일이 삭제됨.')
for file in failed_file_list:
    file_list.remove(file)
    remove_file(file)

100%|██████████| 3238/3238 [24:09<00:00,  2.23it/s]

trimming에 실패한 5개의 파일이 삭제됨.


In [7]:
# 5. 음성이 5초 단위로 분할하여 모두 동일한 길이를 가지도록 조정한다.
split_audio_list = split_audio_list_in_sec(file_list, 5, inplace=True)

# renaming
file_list = rename_files_by_state(data_path)

In [8]:
# 데이터프레임을 생성하여 파일을 분석해보자.
df = pd.DataFrame({'file':file_list})
df['duration'] = df['file'].apply(lambda file: get_duration(file))
df['file'] = df['file'].apply(lambda file: file.rsplit('/', 1)[1])
df['state'] = df['file'].apply(lambda file: file.split('_', 1)[0]).astype('category')

# 모든 파일들은 5초의 값을 가짐을 확인할 수 있다.
# diaper이 596개로 가장 적게 존재하며 sad가 1222개로 가장 많이 존재한다.
count_by_state = df[['file', 'state']].groupby('state').count().reset_index().sort_values('file').reset_index(drop=True)
print(df['duration'].value_counts())
count_by_state

/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


5.0    5828
Name: duration, dtype: int64


,state,file
0,diaper,596
1,uncomfortable,643
2,sleepy,693
3,awake,711
4,hug,844
5,hungry,1119
6,sad,1222


In [9]:
# 6. 음성의 노이즈를 제거한다.

In [14]:
# 7. 음성의 유사도를 분석하여 가장 유사한 n개의 데이터만 사용한다.
# 위 5번 과정을 통해 가장 적은 자료 state의 개수는 596개(diaper)이였음으로 이의 상위 90%인 540개만 사용하겠다.

def delete_dissimilar_wavs(data_path:str, n_limit:int):
  state_list = ['sad', 'hug', 'diaper', 'hungry', 'sleepy', 'awake', 'uncomfortable']
  for state in state_list:
      state_path = os.path.join(data_path, state)
      state_file_list = [os.path.join(state_path, file) for file in os.listdir(state_path)]

      # get similarities
      sim_file_list = get_similarities(state_file_list)

      # 상위 540를 제외하고 모두 제거한다.
      del_file_list = sim_file_list[n_limit:]
      for file in del_file_list:
          remove_file(file)
      print(f'State {state} Done.\n')

delete_dissimilar_wavs(data_path, 540)

Processing: 100%|██████████| 1222/1222 [03:36<00:00,  5.66it/s]


State sad Done.


Processing: 100%|██████████| 844/844 [01:41<00:00,  8.35it/s]


State hug Done.


Processing: 100%|██████████| 596/596 [00:50<00:00, 11.78it/s]


...

State sleepy Done.


Processing: 100%|██████████| 711/711 [01:12<00:00,  9.85it/s]


State awake Done.


Processing: 100%|██████████| 643/643 [00:59<00:00, 10.84it/s]

State uncomfortable Done.


In [4]:
rename_files_by_state(data_path)

['/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_1.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_2.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_3.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_4.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_5.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_6.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_7.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_8.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_9.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_10.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_11.wav',
 '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/sad/sad_12.wav',
 

In [11]:
# 사용을 완료한 info.csv는 제거한다.
remove_file(csv_path)